In [1]:
import org.apache.spark.ml.classification.LogisticRegression
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.linalg.Vectors
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.sql
import org.apache.spark.sql._
import org.apache.spark.sql.types._
import org.apache.spark.ml.classification.LogisticRegression
import org.apache.spark.mllib.linalg.{Vector, Vectors}
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator
import org.apache.spark.ml.feature.{VectorAssembler, StringIndexer}

Intitializing Scala interpreter ...

Spark Web UI available at http://17b13ee3c0e6:4041
SparkContext available as 'sc' (version = 2.4.3, master = local[*], app id = local-1559647293501)
SparkSession available as 'spark'


import org.apache.spark.ml.classification.LogisticRegression
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.linalg.Vectors
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.sql
import org.apache.spark.sql._
import org.apache.spark.sql.types._
import org.apache.spark.ml.classification.LogisticRegression
import org.apache.spark.mllib.linalg.{Vector, Vectors}
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator
import org.apache.spark.ml.feature.{VectorAssembler, StringIndexer}


### Data loading

In [2]:
// Load wrangled datasets from Phase 2: part1.csv; part2.csv; part3.csv

val creditRiskdf1 = spark.read.option("header","true").csv("part1.csv")
val creditRiskdf2 = spark.read.option("header","true").csv("part2.csv")
val creditRiskdf3 = spark.read.option("header","true").csv("part3.csv")

creditRiskdf1: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 283 more fields]
creditRiskdf2: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 283 more fields]
creditRiskdf3: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 283 more fields]


In [3]:
// combine the 3 dataframes to 1
val creditRiskdf1and2 = creditRiskdf1.union(creditRiskdf2)
val creditRiskdf = creditRiskdf1and2.union(creditRiskdf3)

creditRiskdf1and2: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 283 more fields]
creditRiskdf: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 283 more fields]


In [4]:
// check that the rows have been aggregated correctly

println("Total data rows in loaded dataframes:"+(
    creditRiskdf1.count()+creditRiskdf2.count()+creditRiskdf3.count()))

println("Total data rows in combined dataframe:"+creditRiskdf.count())

2019-06-04 11:22:09,620 WARN  [Thread-4] util.Utils (Logging.scala:logWarning(66)) - Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.debug.maxToStringFields' in SparkEnv.conf.
Total data rows in loaded dataframes:183875
Total data rows in combined dataframe:183875


### Data cleansing and reshaping

In [5]:
// Drop features with inconsistent values from dataset.

val incon_droplist = List("previous_loans_DAYS_FIRST_DUE_mean",
                      "previous_loans_SELLERPLACE_AREA_min",
                      "previous_loans_DAYS_FIRST_DUE_sum")

val df = creditRiskdf.drop(incon_droplist:_*)

incon_droplist: List[String] = List(previous_loans_DAYS_FIRST_DUE_mean, previous_loans_SELLERPLACE_AREA_min, previous_loans_DAYS_FIRST_DUE_sum)
df: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 280 more fields]


In [6]:
val creditRiskdf = df

creditRiskdf: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 280 more fields]


In [7]:
//string array of features
val creditRiskFeatures = creditRiskdf.columns

creditRiskFeatures: Array[String] = Array(EXT_SOURCE_2, EXT_SOURCE_3, client_installments_AMT_PAYMENT_min_sum, DAYS_BIRTH, AMT_CREDIT, AMT_ANNUITY, DAYS_EMPLOYED, bureau_DAYS_CREDIT_ENDDATE_max, bureau_DAYS_CREDIT_max, DAYS_ID_PUBLISH, bureau_DAYS_ENDDATE_FACT_max, bureau_AMT_CREDIT_SUM_DEBT_mean, previous_loans_CNT_PAYMENT_mean, client_cash_CNT_INSTALMENT_FUTURE_min_max, previous_loans_SELLERPLACE_AREA_max, DAYS_LAST_PHONE_CHANGE, bureau_AMT_CREDIT_SUM_max, bureau_AMT_CREDIT_SUM_mean, DAYS_REGISTRATION, client_installments_DAYS_INSTALMENT_max_max, previous_loans_AMT_DOWN_PAYMENT_max, CODE_GENDER_F, REGION_POPULATION_RELATIVE, client_cash_CNT_INSTALMENT_max_mean, bureau_DAYS_CREDIT_UPDATE_max, bureau_AMT_CREDIT_SUM_sum, client_installments_NUM_INSTALMENT_VERSION_mean_max, bureau_DAYS_CR...

In [8]:
creditRiskdf.cache()

res1: creditRiskdf.type = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 280 more fields]


In [9]:
//write function to check if there are any outliers in a given domain and show boxplot [Sean]
// Remove Target.

var feat_list = List[String]() //empty list to append features containing outliers in
for (feature <- creditRiskFeatures) {
    //summarise column and query the DataFrame output
    var firstQ = creditRiskdf.select(feature).summary().where(
        $"summary" === "25%").select(feature).first().mkString.toFloat
    var thirdQ = creditRiskdf.select(feature).summary().where(
        $"summary" === "75%").select(feature).first().mkString.toFloat
    //use formulas below to test threshold of outliers
    var testValHigh = thirdQ + (1.5 * (thirdQ - firstQ))
    var testValLow = firstQ - (1.5 * (thirdQ - firstQ))
    //check to see if thresholds are exceeded in the column and count
    var outHigh = creditRiskdf.filter(col(feature) > lit(testValHigh)).count()
    var outLow = creditRiskdf.filter(col(feature) < lit(testValLow)).count()

    //notify us whether or not column contains outliers
    if (outHigh > 0 || outLow > 0 ){
        println(feature.concat(": Contains outliers, #").concat((
            outHigh + outLow).toString))
            feat_list = feature :: feat_list
    }
    //else if (outHigh == 0 || outLow == 0){
        // println(feature.concat(": Does not contain outliers"))
    //}
}

EXT_SOURCE_2: Contains outliers, #755
client_installments_AMT_PAYMENT_min_sum: Contains outliers, #15802
AMT_CREDIT: Contains outliers, #3566
AMT_ANNUITY: Contains outliers, #3756
DAYS_EMPLOYED: Contains outliers, #43216
bureau_DAYS_CREDIT_ENDDATE_max: Contains outliers, #38067
bureau_DAYS_CREDIT_max: Contains outliers, #13523
bureau_DAYS_ENDDATE_FACT_max: Contains outliers, #10731
bureau_AMT_CREDIT_SUM_DEBT_mean: Contains outliers, #16872
previous_loans_CNT_PAYMENT_mean: Contains outliers, #7983
client_cash_CNT_INSTALMENT_FUTURE_min_max: Contains outliers, #20113
previous_loans_SELLERPLACE_AREA_max: Contains outliers, #9178
DAYS_LAST_PHONE_CHANGE: Contains outliers, #59
bureau_AMT_CREDIT_SUM_max: Contains outliers, #15707
bureau_AMT_CREDIT_SUM_mean: Contains outliers, #15610
DAYS_REGISTRATION: Contains outliers, #387
client_installments_DAYS_INSTALMENT_max_max: Contains outliers, #19805
previous_loans_AMT_DOWN_PAYMENT_max: Contains outliers, #13189
REGION_POPULATION_RELATIVE: Contains

previous_loans_CHANNEL_TYPE_Contact center_count_norm: Contains outliers, #27018
bureau_CREDIT_TYPE_Car loan_count_norm: Contains outliers, #14500
client_installments_NUM_INSTALMENT_NUMBER_min_mean: Contains outliers, #16051
previous_loans_WEEKDAY_APPR_PROCESS_START_TUESDAY_count_norm: Contains outliers, #8122
previous_loans_NAME_GOODS_CATEGORY_Furniture_count_norm: Contains outliers, #26363
client_cash_NAME_CONTRACT_STATUS_Completed_count_norm_max: Contains outliers, #9884
AMT_REQ_CREDIT_BUREAU_QRT: Contains outliers, #36296
client_cash_NAME_CONTRACT_STATUS_Active_count_norm_min: Contains outliers, #13971
previous_loans_AMT_DOWN_PAYMENT_min: Contains outliers, #23887
bureau_CREDIT_TYPE_Microloan_count_norm: Contains outliers, #2557
previous_loans_DAYS_TERMINATION_min: Contains outliers, #9044
previous_loans_CHANNEL_TYPE_Channel of corporate sales_count_norm: Contains outliers, #1027
previous_loans_WEEKDAY_APPR_PROCESS_START_SUNDAY_count_norm: Contains outliers, #17199
previous_loans_C

previous_loans_CHANNEL_TYPE_Regional / Local_count: Contains outliers, #44649
ORGANIZATION_TYPE_Transport: type 3: Contains outliers, #616
client_cash_NAME_CONTRACT_STATUS_Returned to the store_count_norm_max: Contains outliers, #2958
ORGANIZATION_TYPE_School: Contains outliers, #5833
previous_loans_NAME_TYPE_SUITE_Other_B_count_norm: Contains outliers, #8856
bureau_CNT_CREDIT_PROLONG_mean: Contains outliers, #6567
client_bureau_balance_MONTHS_BALANCE_max_sum: Contains outliers, #45270
client_cash_NAME_CONTRACT_STATUS_Active_count_min: Contains outliers, #5538
WEEKDAY_APPR_PROCESS_START_WEDNESDAY: Contains outliers, #31521
OCCUPATION_TYPE_High skill tech staff: Contains outliers, #6918
previous_loans_NAME_PAYMENT_TYPE_XNA_count: Contains outliers, #6112
TARGET: Contains outliers, #13312


feat_list: List[String] = List(TARGET, previous_loans_NAME_PAYMENT_TYPE_XNA_count, OCCUPATION_TYPE_High skill tech staff, WEEKDAY_APPR_PROCESS_START_WEDNESDAY, client_cash_NAME_CONTRACT_STATUS_Active_count_min, client_bureau_balance_MONTHS_BALANCE_max_sum, bureau_CNT_CREDIT_PROLONG_mean, previous_loans_NAME_TYPE_SUITE_Other_B_count_norm, ORGANIZATION_TYPE_School, client_cash_NAME_CONTRACT_STATUS_Returned to the store_count_norm_max, ORGANIZATION_TYPE_Transport: type 3, previous_loans_CHANNEL_TYPE_Regional / Local_count, previous_loans_PRODUCT_COMBINATION_POS industry without interest_count_norm, client_installments_NUM_INSTALMENT_NUMBER_max_min, previous_loans_PRODUCT_COMBINATION_Card Street_count_norm, previous_loans_NAME_CONTRACT_STATUS_Refused_count, client_cash_NAME_CONTRACT_STATUS_...

In [10]:
val newlist = feat_list.filter(_!="TARGET")
val df = creditRiskdf.drop(newlist:_*)


newlist: List[String] = List(previous_loans_NAME_PAYMENT_TYPE_XNA_count, OCCUPATION_TYPE_High skill tech staff, WEEKDAY_APPR_PROCESS_START_WEDNESDAY, client_cash_NAME_CONTRACT_STATUS_Active_count_min, client_bureau_balance_MONTHS_BALANCE_max_sum, bureau_CNT_CREDIT_PROLONG_mean, previous_loans_NAME_TYPE_SUITE_Other_B_count_norm, ORGANIZATION_TYPE_School, client_cash_NAME_CONTRACT_STATUS_Returned to the store_count_norm_max, ORGANIZATION_TYPE_Transport: type 3, previous_loans_CHANNEL_TYPE_Regional / Local_count, previous_loans_PRODUCT_COMBINATION_POS industry without interest_count_norm, client_installments_NUM_INSTALMENT_NUMBER_max_min, previous_loans_PRODUCT_COMBINATION_Card Street_count_norm, previous_loans_NAME_CONTRACT_STATUS_Refused_count, client_cash_NAME_CONTRACT_STATUS_Signed_cou...

In [11]:
val creditRiskdf = df
creditRiskdf.cache()
creditRiskdf.schema

creditRiskdf: org.apache.spark.sql.DataFrame = [EXT_SOURCE_3: string, DAYS_BIRTH: string ... 28 more fields]
res3: org.apache.spark.sql.types.StructType = StructType(StructField(EXT_SOURCE_3,StringType,true), StructField(DAYS_BIRTH,StringType,true), StructField(DAYS_ID_PUBLISH,StringType,true), StructField(CODE_GENDER_F,StringType,true), StructField(bureau_DAYS_CREDIT_min,StringType,true), StructField(previous_loans_NAME_YIELD_GROUP_high_count_norm,StringType,true), StructField(previous_loans_DAYS_LAST_DUE_1ST_VERSION_max,StringType,true), StructField(NAME_FAMILY_STATUS_Married,StringType,true), StructField(FLAG_DOCUMENT_3,StringType,true), StructField(previous_loans_NAME_TYPE_SUITE_Unaccompanied_count_norm,StringType,true), StructField(client_installments_DAYS_ENTRY_PAYMENT_max_min,Str...

### Machine learning

In [12]:
var Array(train_df,test_df,validation_df)=creditRiskdf.randomSplit(
    Array(0.7,0.2,0.1))

train_df: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [EXT_SOURCE_3: string, DAYS_BIRTH: string ... 28 more fields]
test_df: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [EXT_SOURCE_3: string, DAYS_BIRTH: string ... 28 more fields]
validation_df: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [EXT_SOURCE_3: string, DAYS_BIRTH: string ... 28 more fields]


In [13]:
val creditRiskFeatures = creditRiskdf.columns
val features = creditRiskFeatures.filter(!_.contains("TARGET"))

creditRiskFeatures: Array[String] = Array(EXT_SOURCE_3, DAYS_BIRTH, DAYS_ID_PUBLISH, CODE_GENDER_F, bureau_DAYS_CREDIT_min, previous_loans_NAME_YIELD_GROUP_high_count_norm, previous_loans_DAYS_LAST_DUE_1ST_VERSION_max, NAME_FAMILY_STATUS_Married, FLAG_DOCUMENT_3, previous_loans_NAME_TYPE_SUITE_Unaccompanied_count_norm, client_installments_DAYS_ENTRY_PAYMENT_max_min, previous_loans_NAME_CONTRACT_STATUS_Approved_count_norm, previous_loans_NAME_PAYMENT_TYPE_XNA_count_norm, previous_loans_CHANNEL_TYPE_Credit and cash offices_count_norm, previous_loans_NAME_PORTFOLIO_POS_count_norm, NAME_INCOME_TYPE_Working, NAME_EDUCATION_TYPE_Secondary / secondary special, previous_loans_NAME_YIELD_GROUP_middle_count_norm, previous_loans_NAME_YIELD_GROUP_XNA_count_norm, previous_loans_NAME_PRODUCT_TYPE_XNA...

In [14]:
val creditRiskFeatures = creditRiskdf.columns
for (colName<-creditRiskFeatures){
     |   train_df=train_df.withColumn(colName,col(colName).cast("Float"))
     | }

creditRiskFeatures: Array[String] = Array(EXT_SOURCE_3, DAYS_BIRTH, DAYS_ID_PUBLISH, CODE_GENDER_F, bureau_DAYS_CREDIT_min, previous_loans_NAME_YIELD_GROUP_high_count_norm, previous_loans_DAYS_LAST_DUE_1ST_VERSION_max, NAME_FAMILY_STATUS_Married, FLAG_DOCUMENT_3, previous_loans_NAME_TYPE_SUITE_Unaccompanied_count_norm, client_installments_DAYS_ENTRY_PAYMENT_max_min, previous_loans_NAME_CONTRACT_STATUS_Approved_count_norm, previous_loans_NAME_PAYMENT_TYPE_XNA_count_norm, previous_loans_CHANNEL_TYPE_Credit and cash offices_count_norm, previous_loans_NAME_PORTFOLIO_POS_count_norm, NAME_INCOME_TYPE_Working, NAME_EDUCATION_TYPE_Secondary / secondary special, previous_loans_NAME_YIELD_GROUP_middle_count_norm, previous_loans_NAME_YIELD_GROUP_XNA_count_norm, previous_loans_NAME_PRODUCT_TYPE_XNA...

In [15]:
train_df.printSchema()

root
 |-- EXT_SOURCE_3: float (nullable = true)
 |-- DAYS_BIRTH: float (nullable = true)
 |-- DAYS_ID_PUBLISH: float (nullable = true)
 |-- CODE_GENDER_F: float (nullable = true)
 |-- bureau_DAYS_CREDIT_min: float (nullable = true)
 |-- previous_loans_NAME_YIELD_GROUP_high_count_norm: float (nullable = true)
 |-- previous_loans_DAYS_LAST_DUE_1ST_VERSION_max: float (nullable = true)
 |-- NAME_FAMILY_STATUS_Married: float (nullable = true)
 |-- FLAG_DOCUMENT_3: float (nullable = true)
 |-- previous_loans_NAME_TYPE_SUITE_Unaccompanied_count_norm: float (nullable = true)
 |-- client_installments_DAYS_ENTRY_PAYMENT_max_min: float (nullable = true)
 |-- previous_loans_NAME_CONTRACT_STATUS_Approved_count_norm: float (nullable = true)
 |-- previous_loans_NAME_PAYMENT_TYPE_XNA_count_norm: float (nullable = true)
 |-- previous_loans_CHANNEL_TYPE_Credit and cash offices_count_norm: float (nullable = true)
 |-- previous_loans_NAME_PORTFOLIO_POS_count_norm: float (nullable = true)
 |-- NAME_INCOME_

In [16]:
val assembler = new VectorAssembler().setInputCols(features).setOutputCol("features")
val df2 = assembler.transform(train_df)

assembler: org.apache.spark.ml.feature.VectorAssembler = vecAssembler_8a51435466a3
df2: org.apache.spark.sql.DataFrame = [EXT_SOURCE_3: float, DAYS_BIRTH: float ... 29 more fields]


In [ ]:
df2.printSchema()

In [17]:
val labelIndexer = new StringIndexer().setInputCol("TARGET").setOutputCol("label")
val df3 = labelIndexer.fit(df2).transform(df2)

labelIndexer: org.apache.spark.ml.feature.StringIndexer = strIdx_8fe42b67193e
df3: org.apache.spark.sql.DataFrame = [EXT_SOURCE_3: float, DAYS_BIRTH: float ... 30 more fields]


In [18]:
val model = new LogisticRegression().fit(df3)
val predictions = model.transform(df3)

2019-06-04 11:28:41,556 WARN  [Thread-4] netlib.BLAS (BLAS.java:<clinit>(61)) - Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
2019-06-04 11:28:41,557 WARN  [Thread-4] netlib.BLAS (BLAS.java:<clinit>(61)) - Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


model: org.apache.spark.ml.classification.LogisticRegressionModel = LogisticRegressionModel: uid = logreg_f6a9b49c9d84, numClasses = 2, numFeatures = 29
predictions: org.apache.spark.sql.DataFrame = [EXT_SOURCE_3: float, DAYS_BIRTH: float ... 33 more fields]


In [19]:
df3.show

+------------+----------+---------------+-------------+----------------------+-----------------------------------------------+--------------------------------------------+--------------------------+---------------+-------------------------------------------------------+----------------------------------------------+-------------------------------------------------------+-----------------------------------------------+--------------------------------------------------------------+--------------------------------------------+------------------------+-------------------------------------------------+-------------------------------------------------+----------------------------------------------+-----------------------------------------------+----------------------------------------------+---------------------------------------------+---------------------------------------------------+-------------------------------------------------------+--------------------------------------------------

|  5.27265E-4|  -14192.0|        -4298.0|          0.0|               -2864.0|                                            0.0|                                    365243.0|                       1.0|            1.0|                                             0.42857143|                                        -160.0|                                             0.21428572|                                     0.64285713|                                                    0.71428573|                                  0.14285715|                     1.0|                                              1.0|                                       0.35714287|                                    0.64285713|                                     0.71428573|                                   0.071428575|                                          0.0|                                          0.9285714|                                             0.78571427|                                         0.1428571

|  5.27265E-4|  -15714.0|        -4945.0|          1.0|               -1650.0|                                            0.5|                                    365243.0|                       1.0|            1.0|                                                    0.0|                                       -1828.0|                                                    0.5|                                           0.25|                                                           0.5|                                        0.25|                     1.0|                                              1.0|                                            0.125|                                          0.25|                                           0.25|                                         0.125|                                         0.25|                                                0.5|                                                    0.5|                                              0.12

In [20]:
predictions.select ("features", "label", "prediction").show(3000)

+--------------------+-----+----------+
|            features|label|prediction|
+--------------------+-----+----------+
|[5.27265015989542...|  0.0|       0.0|
|[5.27265015989542...|  0.0|       0.0|
|[5.27265015989542...|  0.0|       0.0|
|[5.27265015989542...|  1.0|       0.0|
|[5.27265015989542...|  1.0|       0.0|
|[5.27265015989542...|  0.0|       0.0|
|(29,[0,1,2,3,4,6,...|  1.0|       0.0|
|[5.27265015989542...|  0.0|       0.0|
|[5.27265015989542...|  0.0|       0.0|
|[5.27265015989542...|  0.0|       0.0|
|(29,[0,1,2,4,6,7,...|  1.0|       0.0|
|[5.27265015989542...|  1.0|       0.0|
|[5.27265015989542...|  0.0|       0.0|
|[5.27265015989542...|  0.0|       0.0|
|[5.27265015989542...|  0.0|       0.0|
|[5.27265015989542...|  1.0|       0.0|
|[5.27265015989542...|  0.0|       0.0|
|[5.27265015989542...|  0.0|       0.0|
|[5.27265015989542...|  0.0|       0.0|
|[5.27265015989542...|  0.0|       0.0|
|[5.27265015989542...|  0.0|       0.0|
|[5.27265015989542...|  0.0|       0.0|


|[0.13259749114513...|  0.0|       0.0|
|[0.13259749114513...|  0.0|       0.0|
|[0.13259749114513...|  0.0|       0.0|
|[0.13342925906181...|  0.0|       0.0|
|(29,[0,1,2,4,5,6,...|  0.0|       0.0|
|[0.13342925906181...|  0.0|       0.0|
|[0.13342925906181...|  0.0|       0.0|
|[0.13342925906181...|  0.0|       0.0|
|[0.13342925906181...|  0.0|       0.0|
|[0.13426542282104...|  1.0|       0.0|
|[0.13426542282104...|  1.0|       0.0|
|[0.13426542282104...|  0.0|       0.0|
|[0.13426542282104...|  1.0|       0.0|
|[0.13510601222515...|  0.0|       0.0|
|[0.13510601222515...|  0.0|       0.0|
|[0.13510601222515...|  1.0|       0.0|
|[0.13510601222515...|  0.0|       0.0|
|(29,[0,1,2,4,5,6,...|  0.0|       0.0|
|[0.13595104217529...|  0.0|       0.0|
|[0.13595104217529...|  1.0|       0.0|
|[0.13595104217529...|  0.0|       0.0|
|(29,[0,1,2,3,4,6,...|  0.0|       0.0|
|[0.13595104217529...|  0.0|       0.0|
|[0.13680052757263...|  1.0|       0.0|
|[0.13680052757263...|  0.0|       0.0|


|[0.16048893332481...|  0.0|       0.0|
|[0.16146308183670...|  0.0|       0.0|
|(29,[0,1,2,3,4,6,...|  0.0|       0.0|
|(29,[0,1,2,4,6,10...|  0.0|       0.0|
|[0.16146308183670...|  0.0|       0.0|
|[0.16146308183670...|  0.0|       0.0|
|[0.16244199872016...|  0.0|       0.0|
|[0.16244199872016...|  0.0|       0.0|
|[0.16244199872016...|  0.0|       0.0|
|[0.16244199872016...|  1.0|       0.0|
|[0.16244199872016...|  0.0|       0.0|
|[0.16244199872016...|  1.0|       0.0|
|[0.16342569887638...|  1.0|       0.0|
|[0.16342569887638...|  0.0|       0.0|
|(29,[0,1,2,3,4,6,...|  0.0|       0.0|
|[0.16342569887638...|  0.0|       0.0|
|[0.16441418230533...|  1.0|       0.0|
|[0.16441418230533...|  1.0|       0.0|
|[0.16441418230533...|  0.0|       0.0|
|[0.16441418230533...|  0.0|       0.0|
|[0.16441418230533...|  0.0|       0.0|
|[0.16441418230533...|  0.0|       0.0|
|[0.16441418230533...|  0.0|       0.0|
|(29,[0,1,2,3,4,6,...|  0.0|       0.0|
|[0.16540746390819...|  0.0|       0.0|


|[0.19070595502853...|  0.0|       0.0|
|[0.19070595502853...|  0.0|       0.0|
|[0.19070595502853...|  0.0|       0.0|
|[0.19070595502853...|  0.0|       0.0|
|[0.19182160496711...|  0.0|       0.0|
|[0.19182160496711...|  0.0|       0.0|
|[0.19182160496711...|  0.0|       0.0|
|(29,[0,1,2,3,4,6,...|  0.0|       0.0|
|[0.19182160496711...|  0.0|       0.0|
|[0.19182160496711...|  0.0|       0.0|
|[0.19182160496711...|  0.0|       0.0|
|(29,[0,1,2,3,4,6,...|  0.0|       0.0|
|[0.19182160496711...|  0.0|       0.0|
|[0.19182160496711...|  0.0|       0.0|
|[0.19182160496711...|  0.0|       0.0|
|[0.19182160496711...|  0.0|       0.0|
|[0.19294223189353...|  0.0|       0.0|
|[0.19294223189353...|  0.0|       0.0|
|(29,[0,1,2,4,6,7,...|  0.0|       0.0|
|[0.19294223189353...|  0.0|       0.0|
|[0.19294223189353...|  0.0|       0.0|
|[0.19294223189353...|  0.0|       0.0|
|[0.19294223189353...|  1.0|       0.0|
|[0.19406782090663...|  0.0|       0.0|
|[0.19406782090663...|  0.0|       0.0|


|[0.21518240869045...|  0.0|       0.0|
|[0.21518240869045...|  0.0|       0.0|
|[0.21518240869045...|  0.0|       0.0|
|[0.21518240869045...|  0.0|       0.0|
|[0.21518240869045...|  0.0|       0.0|
|(29,[0,1,2,4,6,7,...|  1.0|       0.0|
|(29,[0,1,2,3,4,6,...|  0.0|       0.0|
|(29,[0,1,2,4,6,7,...|  1.0|       0.0|
|[0.21640296280384...|  0.0|       0.0|
|[0.21640296280384...|  1.0|       0.0|
|[0.21640296280384...|  1.0|       0.0|
|[0.21640296280384...|  0.0|       0.0|
|[0.21640296280384...|  0.0|       0.0|
|[0.21640296280384...|  0.0|       0.0|
|[0.21640296280384...|  1.0|       0.0|
|[0.21640296280384...|  0.0|       0.0|
|[0.21640296280384...|  0.0|       0.0|
|(29,[0,1,2,3,4,6,...|  0.0|       0.0|
|[0.21762852370738...|  0.0|       0.0|
|[0.21762852370738...|  0.0|       0.0|
|[0.21762852370738...|  0.0|       0.0|
|[0.21762852370738...|  1.0|       0.0|
|[0.21762852370738...|  0.0|       0.0|
|(29,[0,1,2,3,4,5,...|  0.0|       0.0|
|[0.21762852370738...|  0.0|       0.0|


...

|[0.29558265209198...|  0.0|       0.0|
|[0.29708662629127...|  0.0|       0.0|
|[0.29708662629127...|  0.0|       0.0|
|[0.29708662629127...|  0.0|       0.0|
|[0.29708662629127...|  0.0|       0.0|
|[0.29708662629127...|  0.0|       0.0|
|[0.29708662629127...|  0.0|       0.0|
|[0.29708662629127...|  0.0|       0.0|
|[0.29708662629127...|  0.0|       0.0|
|[0.29708662629127...|  1.0|       0.0|
|[0.29708662629127...|  0.0|       0.0|
|[0.29708662629127...|  0.0|       0.0|
|[0.29708662629127...|  1.0|       0.0|
|[0.29708662629127...|  0.0|       0.0|
|[0.29708662629127...|  0.0|       0.0|
|[0.29708662629127...|  0.0|       0.0|
|[0.29708662629127...|  0.0|       0.0|
|[0.29708662629127...|  0.0|       0.0|
|[0.29708662629127...|  0.0|       0.0|
|[0.29708662629127...|  0.0|       0.0|
|[0.29708662629127...|  0.0|       0.0|
|[0.29859498143196...|  0.0|       0.0|
|[0.29859498143196...|  0.0|       0.0|
|[0.29859498143196...|  0.0|       0.0|
|[0.29859498143196...|  1.0|       0.0|


|(29,[0,1,2,3,4,6,...|  1.0|       0.0|
|[0.31547215580940...|  0.0|       0.0|
|[0.31547215580940...|  0.0|       0.0|
|[0.31547215580940...|  0.0|       0.0|
|[0.31547215580940...|  1.0|       0.0|
|[0.31547215580940...|  0.0|       0.0|
|[0.31547215580940...|  0.0|       0.0|
|[0.31547215580940...|  0.0|       0.0|
|[0.31547215580940...|  0.0|       0.0|
|[0.31547215580940...|  0.0|       0.0|
|[0.31547215580940...|  0.0|       0.0|
|[0.31547215580940...|  0.0|       0.0|
|[0.31547215580940...|  0.0|       0.0|
|[0.31703177094459...|  1.0|       0.0|
|[0.31703177094459...|  1.0|       0.0|
|[0.31703177094459...|  0.0|       0.0|
|[0.31703177094459...|  0.0|       0.0|
|[0.31703177094459...|  0.0|       0.0|
|[0.31703177094459...|  0.0|       0.0|
|[0.31703177094459...|  0.0|       0.0|
|[0.31703177094459...|  0.0|       0.0|
|[0.31703177094459...|  0.0|       0.0|
|[0.31703177094459...|  0.0|       0.0|
|[0.31703177094459...|  0.0|       0.0|
|[0.31703177094459...|  0.0|       0.0|


|[0.34578481316566...|  0.0|       0.0|
|[0.34741821885108...|  0.0|       0.0|
|[0.34741821885108...|  0.0|       0.0|
|[0.34741821885108...|  0.0|       0.0|
|[0.34741821885108...|  0.0|       0.0|
|[0.34741821885108...|  0.0|       0.0|
|(29,[0,1,2,3,4,6,...|  0.0|       0.0|
|[0.34741821885108...|  0.0|       0.0|
|[0.34741821885108...|  0.0|       0.0|
|[0.34741821885108...|  0.0|       0.0|
|[0.34741821885108...|  0.0|       0.0|
|[0.34741821885108...|  0.0|       0.0|
|(29,[0,1,2,4,6,7,...|  0.0|       0.0|
|[0.34741821885108...|  0.0|       0.0|
|(29,[0,1,2,4,6,7,...|  0.0|       0.0|
|[0.34741821885108...|  0.0|       0.0|
|(29,[0,1,2,3,4,6,...|  0.0|       0.0|
|[0.34741821885108...|  0.0|       0.0|
|[0.34741821885108...|  0.0|       0.0|
|[0.34905526041984...|  0.0|       0.0|
|[0.34905526041984...|  0.0|       0.0|
|[0.34905526041984...|  0.0|       0.0|
|[0.34905526041984...|  0.0|       0.0|
|[0.34905526041984...|  0.0|       0.0|
|[0.34905526041984...|  0.0|       0.0|


|[0.36227723956108...|  0.0|       0.0|
|[0.36227723956108...|  0.0|       0.0|
|[0.36227723956108...|  0.0|       0.0|
|(29,[0,1,2,3,4,6,...|  0.0|       0.0|
|[0.36394524574279...|  1.0|       0.0|
|[0.36394524574279...|  0.0|       0.0|
|[0.36394524574279...|  0.0|       0.0|
|[0.36394524574279...|  0.0|       0.0|
|[0.36394524574279...|  0.0|       0.0|
|[0.36394524574279...|  0.0|       0.0|
|[0.36394524574279...|  0.0|       0.0|
|[0.36394524574279...|  0.0|       0.0|
|[0.36394524574279...|  0.0|       0.0|
|[0.36394524574279...|  1.0|       0.0|
|[0.36394524574279...|  0.0|       0.0|
|[0.36394524574279...|  0.0|       0.0|
|(29,[0,1,2,3,4,6,...|  0.0|       0.0|
|[0.36394524574279...|  1.0|       0.0|
|[0.36394524574279...|  0.0|       0.0|
|[0.36561650037765...|  0.0|       0.0|
|[0.36561650037765...|  0.0|       0.0|
|[0.36561650037765...|  0.0|       0.0|
|[0.36561650037765...|  0.0|       0.0|
|(29,[0,1,2,3,4,6,...|  0.0|       0.0|
|[0.36561650037765...|  0.0|       0.0|


|[0.37571102380752...|  0.0|       0.0|
|[0.37571102380752...|  0.0|       0.0|
|[0.37740424275398...|  0.0|       0.0|
|[0.37740424275398...|  0.0|       0.0|
|[0.37740424275398...|  0.0|       0.0|
|[0.37740424275398...|  0.0|       0.0|
|[0.37740424275398...|  0.0|       0.0|
|[0.37740424275398...|  0.0|       0.0|
|[0.37740424275398...|  0.0|       0.0|
|[0.37740424275398...|  0.0|       0.0|
|(29,[0,1,2,3,4,5,...|  0.0|       0.0|
|[0.37740424275398...|  0.0|       0.0|
|[0.37740424275398...|  1.0|       0.0|
|[0.37740424275398...|  0.0|       0.0|
|[0.37740424275398...|  0.0|       0.0|
|[0.37740424275398...|  0.0|       0.0|
|[0.37740424275398...|  1.0|       0.0|
|[0.37740424275398...|  0.0|       0.0|
|[0.37740424275398...|  0.0|       0.0|
|[0.37740424275398...|  1.0|       0.0|
|[0.37740424275398...|  0.0|       0.0|
|(29,[0,1,2,4,5,6,...|  0.0|       0.0|
|[0.37740424275398...|  0.0|       0.0|
|[0.37910047173500...|  0.0|       0.0|
|[0.37910047173500...|  0.0|       0.0|


### Random Forest

In [21]:
// Cross check that the full data set is being used
println("Total data rows in combined dataframe:"+creditRiskdf.count())
var RF_df = creditRiskdf
val creditRiskFeatures = creditRiskdf.columns
for (colName<-creditRiskFeatures){
     |   RF_df=RF_df.withColumn(colName,col(colName).cast("Double"))
     | }
RF_df.printSchema()

Total data rows in combined dataframe:183875
root
 |-- EXT_SOURCE_3: double (nullable = true)
 |-- DAYS_BIRTH: double (nullable = true)
 |-- DAYS_ID_PUBLISH: double (nullable = true)
 |-- CODE_GENDER_F: double (nullable = true)
 |-- bureau_DAYS_CREDIT_min: double (nullable = true)
 |-- previous_loans_NAME_YIELD_GROUP_high_count_norm: double (nullable = true)
 |-- previous_loans_DAYS_LAST_DUE_1ST_VERSION_max: double (nullable = true)
 |-- NAME_FAMILY_STATUS_Married: double (nullable = true)
 |-- FLAG_DOCUMENT_3: double (nullable = true)
 |-- previous_loans_NAME_TYPE_SUITE_Unaccompanied_count_norm: double (nullable = true)
 |-- client_installments_DAYS_ENTRY_PAYMENT_max_min: double (nullable = true)
 |-- previous_loans_NAME_CONTRACT_STATUS_Approved_count_norm: double (nullable = true)
 |-- previous_loans_NAME_PAYMENT_TYPE_XNA_count_norm: double (nullable = true)
 |-- previous_loans_CHANNEL_TYPE_Credit and cash offices_count_norm: double (nullable = true)
 |-- previous_loans_NAME_PORTFOLI

RF_df: org.apache.spark.sql.DataFrame = [EXT_SOURCE_3: double, DAYS_BIRTH: double ... 28 more fields]
creditRiskFeatures: Array[String] = Array(EXT_SOURCE_3, DAYS_BIRTH, DAYS_ID_PUBLISH, CODE_GENDER_F, bureau_DAYS_CREDIT_min, previous_loans_NAME_YIELD_GROUP_high_count_norm, previous_loans_DAYS_LAST_DUE_1ST_VERSION_max, NAME_FAMILY_STATUS_Married, FLAG_DOCUMENT_3, previous_loans_NAME_TYPE_SUITE_Unaccompanied_count_norm, client_installments_DAYS_ENTRY_PAYMENT_max_min, previous_loans_NAME_CONTRACT_STATUS_Approved_count_norm, previous_loans_NAME_PAYMENT_TYPE_XNA_count_norm, previous_loans_CHANNEL_TYPE_Credit and cash offices_count_norm, previous_loans_NAME_PORTFOLIO_POS_count_norm, NAME_INCOME_TYPE_Working, NAME_EDUCATION_TYPE_Secondary / secondary special, previous_loans_NAME_YIELD_GROUP_m...

In [22]:
import org.apache.spark.ml.Pipeline
import org.apache.spark.ml.classification.{RandomForestClassificationModel, RandomForestClassifier}
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator
import org.apache.spark.ml.feature.{IndexToString, StringIndexer, VectorIndexer}

// Load and parse the data file, converting it to a DataFrame.
val data = RF_df
// Index labels, adding metadata to the label column.
// Fit on whole dataset to include all labels in index.
val labelIndexer = new StringIndexer()
  .setInputCol("TARGET")
  .setOutputCol("indexedLabel")
  .fit(data)

import org.apache.spark.ml.Pipeline
import org.apache.spark.ml.classification.{RandomForestClassificationModel, RandomForestClassifier}
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator
import org.apache.spark.ml.feature.{IndexToString, StringIndexer, VectorIndexer}
data: org.apache.spark.sql.DataFrame = [EXT_SOURCE_3: double, DAYS_BIRTH: double ... 28 more fields]
labelIndexer: org.apache.spark.ml.feature.StringIndexerModel = strIdx_42fb7009c6ec


In [23]:
// Automatically identify categorical features, and index them.
// Set maxCategories so features with > 4 distinct values are treated as continuous.
val assembler = new VectorAssembler().setInputCols(features).setOutputCol("features")
val RF_df2 = assembler.transform(RF_df)
val featureIndexer = new VectorIndexer()
  .setInputCol("features")
  .setOutputCol("indexedFeatures")
  .setMaxCategories(4)
  .fit(RF_df2)

assembler: org.apache.spark.ml.feature.VectorAssembler = vecAssembler_bfb7a18911b1
RF_df2: org.apache.spark.sql.DataFrame = [EXT_SOURCE_3: double, DAYS_BIRTH: double ... 29 more fields]
featureIndexer: org.apache.spark.ml.feature.VectorIndexerModel = vecIdx_6ef1f0a62e3a


In [ ]:
RF_df2.show

In [24]:
// Split the data into training and test sets (30% held out for testing).
val Array(trainingData, testData) = RF_df2.randomSplit(Array(0.7, 0.3))

trainingData: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [EXT_SOURCE_3: double, DAYS_BIRTH: double ... 29 more fields]
testData: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [EXT_SOURCE_3: double, DAYS_BIRTH: double ... 29 more fields]


In [25]:
// Train a RandomForest model.
val rf = new RandomForestClassifier()
  .setLabelCol("indexedLabel")
  .setFeaturesCol("indexedFeatures")
  .setNumTrees(20)

rf: org.apache.spark.ml.classification.RandomForestClassifier = rfc_bbe03ff07c45


In [26]:
// Convert indexed labels back to original labels.
val labelConverter = new IndexToString()
  .setInputCol("prediction")
  .setOutputCol("predictedLabel")
  .setLabels(labelIndexer.labels)

labelConverter: org.apache.spark.ml.feature.IndexToString = idxToStr_1b77490b47f9


In [27]:
// Chain indexers and forest in a Pipeline.
val pipeline = new Pipeline()
  .setStages(Array(labelIndexer, featureIndexer, rf, labelConverter))

pipeline: org.apache.spark.ml.Pipeline = pipeline_ed1009422cab


In [28]:
// Train model. This also runs the indexers.
val model = pipeline.fit(trainingData)

model: org.apache.spark.ml.PipelineModel = pipeline_ed1009422cab


In [29]:
// Make predictions.
val predictions = model.transform(testData)

predictions: org.apache.spark.sql.DataFrame = [EXT_SOURCE_3: double, DAYS_BIRTH: double ... 35 more fields]


In [30]:
// Select example rows to display.
predictions.select("predictedLabel", "probability", "features").show(100)


+--------------+--------------------+--------------------+
|predictedLabel|         probability|            features|
+--------------+--------------------+--------------------+
|           0.0|[0.92190895857955...|(29,[0,1,2,3,4,5,...|
|           0.0|[0.92339443898592...|[5.27265E-4,-2149...|
|           0.0|[0.92034841283862...|(29,[0,1,2,3,4,6,...|
|           0.0|[0.92034841283862...|[5.27265E-4,-2099...|
|           0.0|[0.89891655172848...|[5.27265E-4,-2098...|
|           0.0|[0.90043246663635...|[5.27265E-4,-1964...|
|           0.0|[0.90499027162011...|[5.27265E-4,-1954...|
|           0.0|[0.92034841283862...|(29,[0,1,2,3,4,6,...|
|           0.0|[0.91896272090402...|[5.27265E-4,-1898...|
|           0.0|[0.91374964777902...|[5.27265E-4,-1840...|
|           0.0|[0.91343570962215...|[5.27265E-4,-1771...|
|           0.0|[0.91740078383012...|[5.27265E-4,-1765...|
|           0.0|[0.91777331427648...|[5.27265E-4,-1673...|
|           0.0|[0.91791536721669...|[5.27265E-4,-1622..

In [31]:
// Select (prediction, true label) and compute test error.
val evaluator = new MulticlassClassificationEvaluator()
  .setLabelCol("indexedLabel")
  .setPredictionCol("prediction")
  .setMetricName("accuracy")
val accuracy = evaluator.evaluate(predictions)
println(s"Test Error = ${(1.0 - accuracy)}")

Test Error = 0.07298435401314118


evaluator: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_576ff50e288e
accuracy: Double = 0.9270156459868588


In [32]:
val rfModel = model.stages(2).asInstanceOf[RandomForestClassificationModel]
println(s"Learned classification forest model:\n ${rfModel.toDebugString}")

Learned classification forest model:
 RandomForestClassificationModel (uid=rfc_bbe03ff07c45) with 20 trees
  Tree 0 (weight 1.0):
    Predict: 0.0
  Tree 1 (weight 1.0):
    If (feature 18 <= 0.3397435895)
     Predict: 0.0
    Else (feature 18 > 0.3397435895)
     If (feature 0 <= 0.2643481215)
      If (feature 11 <= 0.4913793105)
       If (feature 5 <= 0.102631579)
        Predict: 0.0
       Else (feature 5 > 0.102631579)
        If (feature 1 <= -23630.5)
         Predict: 1.0
        Else (feature 1 > -23630.5)
         Predict: 0.0
      Else (feature 11 > 0.4913793105)
       Predict: 0.0
     Else (feature 0 > 0.2643481215)
      Predict: 0.0
  Tree 2 (weight 1.0):
    If (feature 10 <= -1566.5)
     If (feature 19 <= 0.8284313725)
      If (feature 0 <= 0.302385633)
       If (feature 3 in {0.0})
        If (feature 1 <= -23630.5)
         Predict: 1.0
        Else (feature 1 > -23630.5)
         Predict: 0.0
       Else (feature 3 not in {0.0})
        Predict: 0.0
      El

rfModel: org.apache.spark.ml.classification.RandomForestClassificationModel = RandomForestClassificationModel (uid=rfc_bbe03ff07c45) with 20 trees
